# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse   Shipments                 Customer  \
0  Dec 28 2022  3:05PM  253949        19    60018262  GUSA Granules USA, Inc.   
1  Dec 28 2022  2:55PM  253947        10  0086271535        ISDIN Corporation   
2  Dec 28 2022  2:55PM  253947        10  0086271534        ISDIN Corporation   
3  Dec 28 2022  2:55PM  253947        10  0086271533        ISDIN Corporation   
4  Dec 28 2022  2:55PM  253947        10  0086271383        ISDIN Corporation   
5  Dec 28 2022  2:55PM  253947        10  0086271385        ISDIN Corporation   
6  Dec 28 2022  2:55PM  253947        10  0086271384        ISDIN Corporation   
7  Dec 28 2022  2:55PM  253947        10  0086271390        ISDIN Corporation   
8  Dec 28 2022  2:55PM  253947        10  0086271393        ISDIN Corporation   
9  Dec 28 2022  2:55PM  253947        10  0086271392        ISDIN Corporation   

  ShipmentStatus  
0       Released  
1       Released  
2       Released  
3       Released  
4       Released  
5       Released  
6       Released  
7       Released  
8       Released  
9       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
39  253944       Released          1
40  253945       Released          1
41  253946       Released          3
42  253947       Released         23
43  253949       Released          1

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Executing  Released
id                                 
253944                NaN       1.0
253945                NaN       1.0
253946                NaN       3.0
253947                NaN      23.0
253949                NaN       1.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Executing  Released
id                                 
253862                0.0       1.0
253863                0.0       1.0
253865                0.0      38.0
253868                0.0       3.0
253870                0.0       7.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus  Executing  Released
id                                 
253862                  0         1
253863                  0         1
253865                  0        38
253868                  0         3
253870                  0         7

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Executing  Released
0               253862          0         1
1               253863          0         1
2               253865          0        38
3               253868          0         3
4               253870          0         7

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus      id Executing Released
0               253862                  1
1               253863                  1
2               253865                 38
3               253868                  3
4               253870                  7

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                          Customer
0   Dec 28 2022  3:05PM  253949        19           GUSA Granules USA, Inc.
1   Dec 28 2022  2:55PM  253947        10                 ISDIN Corporation
24  Dec 28 2022  2:52PM  253946        10                  Methapharm, Inc.
27  Dec 28 2022  2:51PM  253945        10                 ISDIN Corporation
28  Dec 28 2022  2:51PM  253944        10      Snap Medical Industries, LLC
29  Dec 28 2022  2:45PM  253943        10      Snap Medical Industries, LLC
32  Dec 28 2022  2:43PM  253939        10                 ISDIN Corporation
55  Dec 28 2022  2:42PM  253942        16  Sartorius Stedim North America  
56  Dec 28 2022  2:36PM  253938        19    Avet Pharmaceuticals Labs, Inc
57  Dec 28 2022  2:27PM  253937        19    Avet Pharmaceuticals Labs, Inc

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                          Customer  \
0  Dec 28 2022  3:05PM  253949        19           GUSA Granules USA, Inc.   
1  Dec 28 2022  2:55PM  253947        10                 ISDIN Corporation   
2  Dec 28 2022  2:52PM  253946        10                  Methapharm, Inc.   
3  Dec 28 2022  2:51PM  253945        10                 ISDIN Corporation   
4  Dec 28 2022  2:51PM  253944        10      Snap Medical Industries, LLC   
5  Dec 28 2022  2:45PM  253943        10      Snap Medical Industries, LLC   
6  Dec 28 2022  2:43PM  253939        10                 ISDIN Corporation   
7  Dec 28 2022  2:42PM  253942        16  Sartorius Stedim North America     
8  Dec 28 2022  2:36PM  253938        19    Avet Pharmaceuticals Labs, Inc   
9  Dec 28 2022  2:27PM  253937        19    Avet Pharmaceuticals Labs, Inc   

  Executing Released  
0                  1  
1                 23  
2                  3  
3                  1  
4                  1  
5                  3  
6                 23  
7                  1  
8                  1  
9                  1

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                          Customer  \
0  Dec 28 2022  3:05PM  253949        19           GUSA Granules USA, Inc.   
1  Dec 28 2022  2:55PM  253947        10                 ISDIN Corporation   
2  Dec 28 2022  2:52PM  253946        10                  Methapharm, Inc.   
3  Dec 28 2022  2:51PM  253945        10                 ISDIN Corporation   
4  Dec 28 2022  2:51PM  253944        10      Snap Medical Industries, LLC   
5  Dec 28 2022  2:45PM  253943        10      Snap Medical Industries, LLC   
6  Dec 28 2022  2:43PM  253939        10                 ISDIN Corporation   
7  Dec 28 2022  2:42PM  253942        16  Sartorius Stedim North America     
8  Dec 28 2022  2:36PM  253938        19    Avet Pharmaceuticals Labs, Inc   
9  Dec 28 2022  2:27PM  253937        19    Avet Pharmaceuticals Labs, Inc   

  Released Executing  
0        1            
1       23            
2        3            
3        1            
4        1            
5        3            
6       23            
7        1            
8        1            
9        1

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                      Customer  \
0  Dec 28 2022  3:05PM  253949        19       GUSA Granules USA, Inc.   
1  Dec 28 2022  2:55PM  253947        10             ISDIN Corporation   
2  Dec 28 2022  2:52PM  253946        10              Methapharm, Inc.   
3  Dec 28 2022  2:51PM  253945        10             ISDIN Corporation   
4  Dec 28 2022  2:51PM  253944        10  Snap Medical Industries, LLC   

  Released Executing  
0        1            
1       23            
2        3            
3        1            
4        1

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


Date      id Warehouse                      Customer  \
0  Dec 28 2022  3:05PM  253949        19       GUSA Granules USA, Inc.   
1  Dec 28 2022  2:55PM  253947        10             ISDIN Corporation   
2  Dec 28 2022  2:52PM  253946        10              Methapharm, Inc.   
3  Dec 28 2022  2:51PM  253945        10             ISDIN Corporation   
4  Dec 28 2022  2:51PM  253944        10  Snap Medical Industries, LLC   

   Released  Executing  
0       1.0        NaN  
1      23.0        NaN  
2       3.0        NaN  
3       1.0        NaN  
4       1.0        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                      Customer  \
0  Dec 28 2022  3:05PM  253949        19       GUSA Granules USA, Inc.   
1  Dec 28 2022  2:55PM  253947        10             ISDIN Corporation   
2  Dec 28 2022  2:52PM  253946        10              Methapharm, Inc.   
3  Dec 28 2022  2:51PM  253945        10             ISDIN Corporation   
4  Dec 28 2022  2:51PM  253944        10  Snap Medical Industries, LLC   

   Released  Executing  
0       1.0        0.0  
1      23.0        0.0  
2       3.0        0.0  
3       1.0        0.0  
4       1.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing
Warehouse                              
10         8632914     304.0        0.0
16          253942       1.0        0.0
19         2285258      30.0        1.0
21          253916       1.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing
0        10  8632914     304.0        0.0
1        16   253942       1.0        0.0
2        19  2285258      30.0        1.0
3        21   253916       1.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing
0        10     304.0        0.0
1        16       1.0        0.0
2        19      30.0        1.0
3        21       1.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse     Status  Value
0        10   Released  304.0
1        16   Released    1.0
2        19   Released   30.0
3        21   Released    1.0
4        10  Executing    0.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse     10   16    19   21
Status                          
Executing    0.0  0.0   1.0  0.0
Released   304.0  1.0  30.0  1.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status     10   16    19   21
0          Executing    0.0  0.0   1.0  0.0
1           Released  304.0  1.0  30.0  1.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status     10   16    19   21
0  Executing    0.0  0.0   1.0  0.0
1   Released  304.0  1.0  30.0  1.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()